In [8]:
# # Downloading the dataset - Run When using GitHub, as the download path is based on GitHUb CodeSpaces
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet -P /workspaces/mlops-learning/01-intro/data
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet -P /workspaces/mlops-learning/01-intro/data


# # Download 2021 data
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet -P /workspaces/mlops-learning/01-intro/data
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet -P /workspaces/mlops-learning/01-intro/data

# When Running on Other Platform
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet -P ../01-intro/data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet -P ../01-intro/data


# Download 2021 data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet -P ../01-intro/data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet -P ../01-intro/data

--2024-08-30 08:07:35--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.130, 13.226.36.73, 13.226.36.218, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49961641 (48M) [binary/octet-stream]
Saving to: ‘../01-intro/data/yellow_tripdata_2024-01.parquet’

yellow_tripdata_202 100%[===================>]  47.65M  --.-KB/s    in 0.1s    

2024-08-30 08:07:35 (346 MB/s) - ‘../01-intro/data/yellow_tripdata_2024-01.parquet’ saved [49961641/49961641]



--2024-08-30 08:07:35--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.130, 13.226.36.73, 13.226.36.196, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50349284 (48M) [binary/octet-stream]
Saving to: ‘../01-intro/data/yellow_tripdata_2024-02.parquet’

yellow_tripdata_202 100%[===================>]  48.02M  --.-KB/s    in 0.1s    

2024-08-30 08:07:35 (410 MB/s) - ‘../01-intro/data/yellow_tripdata_2024-02.parquet’ saved [50349284/50349284]

--2024-08-30 08:07:35--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.73, 13.226.36.130, 13.226.36.218, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.73

To check the indepth version of Training and Prediction, check 01-intro folder

In [1]:
# importing all the necesasry libraries

import pickle
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso

from sklearn.metrics import root_mean_squared_error


from sklearn.model_selection import cross_val_score, KFold

We start by Setting the Tracking URI

In [2]:
import mlflow

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# run only one from the below and comment the other

# Create a new MLflow Experiment - Inside an experiment, there will be Runs
# mlflow.set_experiment("taxi-prediction-xgboost-hyperopt")

# Creating a new one for AutoLog, so can see the with and without AutoLog
mlflow.set_experiment("taxi-prediction-xgboost-hyperopt-with-autolog")

<Experiment: artifact_location='mlflow-artifacts:/114282133553756653', creation_time=1725019412855, experiment_id='114282133553756653', last_update_time=1725019412855, lifecycle_stage='active', name='taxi-prediction-xgboost-hyperopt-with-autolog', tags={}>

In [3]:
# a function to read the data, preprocess it and return it
def read_and_preprocess(filename):
    data = pd.read_parquet(filename)
    
    # create the target variable
    data['ride_duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime'] 
    data['ride_duration'] = data['ride_duration'].apply(lambda x: x.total_seconds()/60) 

    # take only the data below 1 hour
    data = data[(data['ride_duration'] >= 1) & (data['ride_duration'] <= 60)]

    # # sample the data to 70k rows
    # if len(data) > 70000:
    #     sampled_data = data.iloc[:70000,:].copy()
    # else:
    #     sampled_data = data.copy()
    sampled_data = data.copy()
    
    # chosing categorical
    categorical = ['PULocationID', 'DOLocationID']

    # convert these numerical categorical features to string categorical features
    sampled_data[categorical] = sampled_data[categorical].astype(str)


    return sampled_data

In [4]:
# When using GitHub CodeSpaces - Path is Set according to that
# df_train = read_and_preprocess('/workspaces/mlops-learning/01-intro/data/yellow_tripdata_2021-01.parquet')
# df_valid = read_and_preprocess('/workspaces/mlops-learning/01-intro/data/yellow_tripdata_2021-02.parquet')

# when not using GitHub CodeSpaces
df_train = read_and_preprocess('../01-intro/data/yellow_tripdata_2021-01.parquet')
df_valid = read_and_preprocess('../01-intro/data/yellow_tripdata_2021-02.parquet')

In [5]:
# chosing categorical and numerical features
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# to use the DictVectorizer, we need to convert the dataframe to dict
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_valid[categorical + numerical].to_dict(orient='records')


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_valid = dv.fit_transform(val_dicts)

# storing our target variable
target = 'ride_duration'
y_train = df_train[target].values
y_val = df_valid[target].values

Using XGBoost

In [6]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [7]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_valid, label=y_val)

In [9]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

The below code does not use AutoLog. For AutoLog, skip this step

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# so the objective will be called 10 times and for each time, a run will be registered in the MLFlow
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/mlops-learning/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:33:36] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.76296                           
[1]	validation-rmse:8.28224                           
[2]	validation-rmse:7.84964                           
[3]	validation-rmse:7.46061                           
[4]	validation-rmse:7.11444                           
[5]	validation-rmse:6.80408                           
[6]	validation-rmse:6.52773                           
[7]	validation-rmse:6.28252                           
[8]	validation-rmse:6.06369                           
[9]	validation-rmse:5.86982                           
[10]	validation-rmse:5.70158                          
[11]	validation-rmse:5.55365                          
[12]	validation-rmse:5.42534                          
[13]	validation-rmse:5.31378                          
[14]	validation-rmse:5.21564                          
[15]	validation-rmse:5.13104                          
[16]	validation-rmse:5.05841                          
[17]	validation-rmse:4.99481                          
[18]	valid

2024/08/30 10:38:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-whale-158 at: http://127.0.0.1:5000/#/experiments/107471247143277881/runs/0a741829cadb4384993cb10a25f4e486.

2024/08/30 10:38:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/107471247143277881.



 10%|█         | 1/10 [04:54<44:11, 294.65s/trial, best loss: 4.67849580384672]

...

 90%|█████████ | 9/10 [15:31<02:13, 133.42s/trial, best loss: 4.504647348518104]

/root/mlops-learning/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:49:08] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.49508                                                     
[1]	validation-rmse:5.27101                                                     
[2]	validation-rmse:4.81807                                                     
[3]	validation-rmse:4.66455                                                     
[4]	validation-rmse:4.60846                                                     
[5]	validation-rmse:4.59279                                                     
[6]	validation-rmse:4.57714                                                     
[7]	validation-rmse:4.56750                                                     
[8]	validation-rmse:4.56758                                                     
[9]	validation-rmse:4.57087                                                     
[10]	validation-rmse:4.57360                                                    
[11]	validation-rmse:4.57406                                                    
[12]	validation-rmse:4.57330

2024/08/30 10:49:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run enchanting-bug-95 at: http://127.0.0.1:5000/#/experiments/107471247143277881/runs/f609759083824907a1388169c733fbd8.

2024/08/30 10:49:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/107471247143277881.



100%|██████████| 10/10 [15:48<00:00, 94.83s/trial, best loss: 4.504647348518104]


We can also use the Built-in Auto Logging Provided by MLFlow for different models

Here, we are testing for a single example. 

Note: AutoLogging XGBoost Fails for Some Reason

In [1]:
# # these parameters are taken after running the above cell and
# # checking which parameters got the best rmse
# params = {
#     'max_depth': 36,
#     'learning_rate': 0.4,
#     'reg_alpha': 0.015,
#     'reg_lambda': 0.0053,
#     'min_child_weight': 15.88,
#     'seed': 42
# }

# # we provide the autolog here, just before the run function which gets executed in objective function
# mlflow.xgboost.autolog() 


# booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=100,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=10
#         )
